In [1]:
############################
# 1) Install Dependencies #
############################
!pip install --upgrade torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade transformers datasets evaluate librosa seaborn
!pip install bitsandbytes
!pip install pydub



Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:

############################
# 2) Import Libraries     #
############################
import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:true"
import requests
import shutil
from zipfile import ZipFile
import torch
import librosa
import datasets
import evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_from_disk, Dataset, DatasetDict, concatenate_datasets
from transformers import (
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2Processor,
    TrainingArguments,
    Trainer
)
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from bitsandbytes.optim import Adam8bit
from sklearn.metrics import classification_report, confusion_matrix
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
############################
# 3) Check GPU & Versions #
############################
print("Torch version:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("GPU available?", torch.cuda.is_available())
print("Using device:", device)
!nvidia-smi


Torch version: 2.6.0+cu118
GPU available? True
Using device: cuda
Mon Feb 24 15:13:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |      

## Data acquisition

In [3]:
urls = [
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc2.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc3.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc4.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc5.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc6.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc7.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc8.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc9.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc10.zip",
    "https://ars.els-cdn.com/content/image/1-s2.0-S0885230821001340-mmc11.zip"
]

# 2) SET UP FOLDERS
# Create root directories for your final data
os.makedirs("Healthy", exist_ok=True)
os.makedirs("MCI", exist_ok=True)
os.makedirs("AD", exist_ok=True)

# Create a temporary folder for extracted files
temp_folder = "tmp_extracted"
os.makedirs(temp_folder, exist_ok=True)

# 3) DOWNLOAD AND EXTRACT EACH ZIP
for i, url in enumerate(urls):
    zip_filename = f"downloaded_{i}.zip"   # A local name to store the downloaded file

    # Download the file
    print(f"Downloading from {url}...")
    response = requests.get(url)
    with open(zip_filename, "wb") as f:
        f.write(response.content)
    print(f"Saved {zip_filename}")

    # Extract all contents into the temp_folder
    print(f"Extracting {zip_filename}...")
    with ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(temp_folder)

    # Optionally delete the ZIP file after extraction to save space
    os.remove(zip_filename)

temp_folder = "tmp_extracted"  # your temporary extraction folder



# 4) MOVE FILES INTO THE RIGHT FOLDERS (WITH MP3 TO WAV CONVERSION)
for root, dirs, files in os.walk(temp_folder):
    for filename in files:
        full_path = os.path.join(root, filename)

        # Convert MP3 to WAV if needed
        if filename.endswith(".mp3"):
            wav_filename = filename.replace(".mp3", ".wav")
            wav_path = os.path.join(root, wav_filename)

            # Convert MP3 to WAV
            audio = AudioSegment.from_mp3(full_path)
            audio.export(wav_path, format="wav")

            # Remove the original MP3
            os.remove(full_path)

            # Update full_path to the new WAV file
            full_path = wav_path
            filename = wav_filename

        # Move to corresponding folder
        if filename.startswith("AD"):
            shutil.move(full_path, os.path.join("AD", filename))
        elif filename.startswith("MCI"):
            shutil.move(full_path, os.path.join("MCI", filename))
        elif filename.startswith("HC"):
            shutil.move(full_path, os.path.join("Healthy", filename))
        else:
            print(f"File '{filename}' doesn't match AD/MCI/HC. Skipping or placing it elsewhere.")

print("MP3 conversion and file moving completed.")


# 5) CLEAN UP
shutil.rmtree(temp_folder, ignore_errors=True)
print("Temporary folder removed.")


Saved downloaded_0.zip
Extracting downloaded_0.zip...
Saved downloaded_1.zip
Extracting downloaded_1.zip...
Saved downloaded_2.zip
Extracting downloaded_2.zip...
Saved downloaded_3.zip
Extracting downloaded_3.zip...
Saved downloaded_4.zip
Extracting downloaded_4.zip...
Saved downloaded_5.zip
Extracting downloaded_5.zip...
Saved downloaded_6.zip
Extracting downloaded_6.zip...
Saved downloaded_7.zip
Extracting downloaded_7.zip...
Saved downloaded_8.zip
Extracting downloaded_8.zip...
Saved downloaded_9.zip
Extracting downloaded_9.zip...
MP3 conversion and file moving completed.
Temporary folder removed.


In [5]:

############################
# 1) Data Preprocessing   #
############################
from transformers.modeling_outputs import SequenceClassifierOutput
#modifiedd model that adds prosodic features
class Wav2Vec2ProsodicClassifier(nn.Module):
    def __init__(self, base_model, num_labels, prosodic_dim=3):
        super().__init__()
        self.wav2vec2 = base_model  # Wav2Vec2ForSequenceClassification base model
        self.fc_prosody = nn.Linear(prosodic_dim, 16)  # Project prosodic features
        # Dynamically get the hidden size from the base model configuration
        hidden_size = self.wav2vec2.config.hidden_size
        self.fc_combined = nn.Linear(hidden_size + 16, num_labels)  # Combine Wav2Vec2 + prosody

    def forward(self, input_values, prosodic_features, attention_mask=None, labels=None, **kwargs):
      outputs = self.wav2vec2(
          input_values,
          attention_mask=attention_mask,
          output_hidden_states=True
      )
      wav_embeddings = outputs.hidden_states[-1].mean(dim=1)  # (Batch, hidden_size)
      prosodic_embeddings = self.fc_prosody(prosodic_features)  # (Batch, 16)
      combined_features = torch.cat((wav_embeddings, prosodic_embeddings), dim=-1)
      logits = self.fc_combined(combined_features)  # (Batch, num_labels)

      # Ensure logits always have a batch dimension
      if logits.dim() == 1:
          logits = logits.unsqueeze(0)

      loss = None
      if labels is not None:
          loss_fct = nn.CrossEntropyLoss()
          loss = loss_fct(logits, labels)

      return SequenceClassifierOutput(
          loss=loss,
          logits=logits,
          hidden_states=outputs.hidden_states,
          attentions=outputs.attentions
      )


    def gradient_checkpointing_enable(self):
        self.wav2vec2.gradient_checkpointing_enable()


model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-spanish"
processor = Wav2Vec2Processor.from_pretrained(model_name)
base_model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-spanish and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:


#Sample weights calculations to compensate for imbalancedd dataset
num_samples_per_class = {
    0: 197,  # Healthy (HC)
    1: 90,   # MCI
    2: 74    # AD
}

total_samples = sum(num_samples_per_class.values())
num_classes = len(num_samples_per_class)
# Compute class weights dynamically
class_weights = {cls: total_samples / (num_classes * count) for cls, count in num_samples_per_class.items()}
# Convert to PyTorch tensor
device = "cuda" if torch.cuda.is_available() else "cpu"
weights_tensor = torch.tensor([class_weights[0], class_weights[1], class_weights[2]], dtype=torch.float).to(device)
# Use it in CrossEntropyLoss
criterion = torch.nn.CrossEntropyLoss(weight=weights_tensor)



ROOT_DIR = os.getcwd()
DATASET_PATH = ROOT_DIR  # Root folder where audio files are stored
OUTPUT_PATH = os.path.join(ROOT_DIR, "ProcessedFiles")
os.makedirs(OUTPUT_PATH, exist_ok=True)

LABEL_MAP = {"Healthy": 0, "MCI": 1, "AD": 2}

def load_audio(file_path, target_sr=16000):
    audio, sr = librosa.load(file_path, sr=target_sr)
    return np.array(audio, dtype=np.float32), sr  # Ensure float32 output

def preprocess_function(example):
    """Processes Wav2Vec2 inputs and extracts prosodic features."""
    audio = example["audio"]

    # Wav2Vec2 processing
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")

    # Extract prosodic features (without pitch)
    prosodic_features = extract_prosodic_features(example["audio"]["path"])

    # Convert prosodic features to tensor
    inputs["prosodic_features"] = torch.tensor(prosodic_features, dtype=torch.float32)
    inputs["label"] = example["label"]

    return inputs

def chunk_audio(example, max_length=16000*30):  # 30 seconds max
    audio = example["audio"]
    if len(audio) > max_length:
        example["audio"] = audio[:max_length]
    return preprocess_function(example)


In [7]:

audio_files = []
labels = []

for category in LABEL_MAP.keys():
    category_path = os.path.join(DATASET_PATH, category)
    if not os.path.exists(category_path):
        continue
    for file in os.listdir(category_path):
        if file.endswith(".wav"):
            audio_files.append(os.path.join(category_path, file))
            labels.append(LABEL_MAP[category])

data_df = pd.DataFrame({"file_path": audio_files, "label": labels})

def extract_prosodic_features(audio_path):
    """Extracts prosodic features: pauses and speech rate (no pitch)."""
    y, sr = librosa.load(audio_path, sr=16000)
    # Compute energy for silence detection
    energy = librosa.feature.rms(y=y)
    silence_threshold = np.percentile(energy, 10)  # 10th percentile as threshold
    pauses = np.where(energy < silence_threshold)[1]
    num_pauses = len(pauses)
    total_pause_duration = len(pauses) / sr  # Convert to seconds
    # Speech rate (words per second)
    speech_rate = librosa.beat.tempo(y=y, sr=sr)[0] / 60  # Convert BPM to words per second
    return np.array([num_pauses, total_pause_duration, speech_rate], dtype=np.float32)



# Splitting dataset
train_files, test_files, train_labels, test_labels = train_test_split(
    data_df["file_path"], data_df["label"],
    test_size=0.2, stratify=data_df["label"], random_state=42
)
train_files, val_files, train_labels, val_labels = train_test_split(
    train_files, train_labels,
    test_size=0.1, stratify=train_labels, random_state=42
)

def process_data(files, labels):
    data = []
    for file, label in tqdm(zip(files, labels), total=len(files)):
        audio, sr = load_audio(file)
        data.append({
            "audio": {
                "array": np.array(audio, dtype=np.float32),
                "sampling_rate": sr,
                "path": file  # Save file path for prosodic feature extraction if needed
            },
            "label": label
        })
    return data

train_data = process_data(train_files, train_labels)
val_data = process_data(val_files, val_labels)
test_data = process_data(test_files, test_labels)

dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data),
    "test": Dataset.from_list(test_data),
})

dataset = dataset.map(chunk_audio)
dataset.save_to_disk(OUTPUT_PATH)
print(f"Dataset saved to {OUTPUT_PATH}")



100%|██████████| 73/73 [00:04<00:00, 18.15it/s]


Map:   0%|          | 0/259 [00:00<?, ? examples/s]

<ipython-input-7-ec746fc415a3>:25: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  speech_rate = librosa.beat.tempo(y=y, sr=sr)[0] / 60  # Convert BPM to words per second


Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/259 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/29 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/73 [00:00<?, ? examples/s]

Dataset saved to /content/ProcessedFiles


In [8]:
############################
# 2) Load Dataset          #
############################

dataset = load_from_disk(OUTPUT_PATH)


In [ ]:
dataset

In [9]:
############################
# 2) Load Model            #
############################
model = Wav2Vec2ProsodicClassifier(base_model, num_labels=3)
model.gradient_checkpointing_enable()
optimizer = Adam8bit(model.parameters(), lr=2e-5)



In [10]:


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    report = classification_report(labels, predictions, target_names=["Healthy", "MCI", "AD"], output_dict=True)

    print("Classification Report:\n", classification_report(labels, predictions, target_names=["Healthy", "MCI", "AD"]))

    return {
        "accuracy": report["accuracy"],
        "f1_macro": report["macro avg"]["f1-score"],
        "f1_healthy": report["Healthy"]["f1-score"],
        "f1_mci": report["MCI"]["f1-score"],
        "f1_ad": report["AD"]["f1-score"],
    }

############################
# 3) Data Collator         #
############################

def data_collator(features):
    waveforms = [torch.tensor(f["audio"]["array"]) for f in features]
    prosodic_features = torch.stack([
        torch.tensor(f["prosodic_features"], dtype=torch.float) for f in features
    ])  # Now each prosodic_features is converted to a tensor
    labels = torch.tensor([f["label"] for f in features])

    input_values = pad_sequence(waveforms, batch_first=True, padding_value=0)

    inputs = processor(
        input_values.numpy(),
        sampling_rate=16000,
        padding=True,
        return_tensors="pt"
    )

    inputs["labels"] = labels
    inputs["prosodic_features"] = prosodic_features  # Add prosodic features

    return inputs



############################
# 4) Training Arguments   #
############################

training_args = TrainingArguments(
    output_dir="./wav2vec2_classification",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    fp16=True,
    remove_unused_columns=False,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        prosodic_features = inputs.pop("prosodic_features")  # Extract prosodic features

        outputs = model(input_values=inputs["input_values"], prosodic_features=prosodic_features, labels=labels)  # Forward pass
        logits = outputs.logits  # Extract logits

        loss_fct = nn.CrossEntropyLoss()  # Class-weighted loss can be added here if needed
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only=False, ignore_keys=None):
        labels = inputs.pop("labels", None)  # Remove labels from inputs
        prosodic_features = inputs.pop("prosodic_features")

        with torch.no_grad():
            outputs = model(input_values=inputs["input_values"], prosodic_features=prosodic_features, labels=labels)

        loss = outputs.loss if outputs.loss is not None else None
        logits = outputs.logits

        if prediction_loss_only:
            return (loss, None, None)

        return (loss, logits, labels)


# Update Trainer initialization
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    optimizers=(optimizer, None)
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Healthy,F1 Mci,F1 Ad
1,8.578900,1.766718,0.344828,0.246677,0.315789,0.424242,0.000000
2,4.275800,1.136491,0.655172,0.442577,0.857143,0.470588,0.000000
3,3.909000,1.064428,0.586207,0.358396,0.789474,0.285714,0.000000
4,4.376500,1.037484,0.586207,0.358396,0.789474,0.285714,0.000000


Classification Report:
               precision    recall  f1-score   support

     Healthy       1.00      0.19      0.32        16
         MCI       0.27      1.00      0.42         7
          AD       0.00      0.00      0.00         6

    accuracy                           0.34        29
   macro avg       0.42      0.40      0.25        29
weighted avg       0.62      0.34      0.28        29



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Classification Report:
               precision    recall  f1-score   support

     Healthy       0.79      0.94      0.86        16
         MCI       0.40      0.57      0.47         7
          AD       0.00      0.00      0.00         6

    accuracy                           0.66        29
   macro avg       0.40      0.50      0.44        29
weighted avg       0.53      0.66      0.59        29



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

...

Classification Report:
               precision    recall  f1-score   support

     Healthy       0.68      0.94      0.79        16
         MCI       0.29      0.29      0.29         7
          AD       0.00      0.00      0.00         6

    accuracy                           0.59        29
   macro avg       0.32      0.41      0.36        29
weighted avg       0.45      0.59      0.50        29



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Classification Report:
               precision    recall  f1-score   support

     Healthy       0.68      0.94      0.79        16
         MCI       0.29      0.29      0.29         7
          AD       0.00      0.00      0.00         6

    accuracy                           0.59        29
   macro avg       0.32      0.41      0.36        29
weighted avg       0.45      0.59      0.50        29



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

AttributeError: 'Wav2Vec2ProsodicClassifier' object has no attribute 'save_pretrained'

In [13]:

torch.save(model.state_dict(), "./wav2vec2_classification/model.pth")
processor.save_pretrained("./wav2vec2_classification")

print("Training complete! Model saved to ./wav2vec2_classification")


Training complete! Model saved to ./wav2vec2_classification
